In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from datetime import datetime
import itertools
from tqdm import tqdm
import torch
import geopandas as gpd

In [1]:
import re
with open('SpaceTempralconflict.ipynb', 'r') as file:
    # Read original notebook file to string
    jupyter = file.read()

    # Run a regex based search and replace, wipe all empty outputs properties.
    outputs_tag_removed = re.sub(',\n\s+\"outputs\":\ \[\]', '', jupyter)

    # Overwrite original notebook file
    print(outputs_tag_removed, file=open('SpaceTempralconflict.ipynb', 'w'))

In [ ]:
cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
frontiers = gpd.read_file((gpd.datasets.get_path('naturalearth_lowres')))

In [ ]:
cities.head()

In [ ]:
frontiers.head()

In [ ]:
df_events=pd.read_csv('ged201.csv' ,dtype={'gwnoa': object})

In [ ]:
df_events.info()

In [ ]:
print(f'{len(df_events.latitude)/len (df_events)*100}%')

In [ ]:
print(f'{len(df_events.longitude)/len (df_events)*100}%')


In [ ]:
serie_noNan=df_events.isna().sum()==0

In [ ]:
df_nonNan=pd.DataFrame(serie_noNan, columns=['valid'])

In [ ]:
df_nonNan.index

In [ ]:
df_nonNan = df_nonNan[df_nonNan.valid==True]

In [ ]:
df_nonNan.index

In [ ]:
df_dataset= df_events[df_nonNan.index]

In [ ]:
df_dataset

In [ ]:
df_dataset.info()

fig, (ax1) = plt.subplots(nrows=1, ncols=1, figsize=(15, 5))
plt.suptitle(f'densité des zones de conflit par état ', fontsize=16)
ax1.pie(df_dataset.country.value_counts(),labels=df_dataset.country.value_counts().index)


In [ ]:
gdf.region.unique()

# Type of violence for year 2014 by region
## Purple: state-based conflict, 
## Sky-blue: non-state conflict, 
## Brown: one-sided violence

In [ ]:

Year_2002=gdf[gdf.year==2002]

In [ ]:
 def hot_point_by_year(year):
    gdf = gpd.GeoDataFrame(
        df_dataset, geometry=gpd.points_from_xy(df_dataset.longitude, df_dataset.latitude), crs="EPSG:4326"
    )
    year_hot_point=gdf[gdf.year==year]
    for region in gdf.region.unique():
        fig, ax = plt.subplots(figsize=(10, 10))
        gdf.plot(ax=ax)
        
        xmin, ymin, xmax, ymax = gdf[gdf.region == region].total_bounds
        pad = 5  # add a padding around the geometry
        ax.set_xlim(xmin-pad, xmax+pad)
        ax.set_ylim(ymin-pad, ymax+pad)
        # We can now plot our ``GeoDataFrame``.
        frontiers.plot(ax=ax)
        year_hot_point.plot(ax=ax, column="type_of_violence", legend=True) 
        plt.title(f'Region: {region}, Année: {year}')

In [ ]:
hot_point_by_year(2014)

In [ ]:
hot_point_by_year(2015)

# ZONE d'EXPLOITATION DU DATASET
- une frappe pour un couple latitude longitude20
- intensité du conflit nombre de décès
- intensité de la frappe de faible à forte
- suivre  le dataset en time series
- predire la probabilité d'une frappe et de son intensité sur quelle latitude longitude dans un laps de temps donné.


# Zone de reflexion sur le model
- Graphe de connexion 
- noeud A tuple (start end) avec features ne définissant pas les résultantes de l'attaques
- noeud B tuple (longitude latitude) avec features définissant les résultantes de l'attaque
- lien edge entre A et B
- layers d'attention (GATConv ou GAT2vConv ou GraphSAGE)
- layers GCN? 2?
- definir un nombre fini des cinq (ou dix mais definis) prochaines attaques quand? quelles ? intensité?
- la définition se fera sur la plus grande probabilité parmi les prédicition
- metriques ROC AUC pour la fiabilité.

#  Zone de reflexion sur un graphe
- regression linéaire
- autant de edgess qu'il y a de lignes
- il n'y a "que" 11312 date de debut d'events sur 225385 enregistrements
- on anticipe un graphe à 11312 edges

In [ ]:
df_dataset['date_start'].describe()

G = nx.Graph()
for node_tuple in [df_dataset.date_start,df_dataset.longitude, df_dataset.latitude, df_dataset.type_of_violence]:
    G.add_edges_from(itertools.product(node_tuple, node_tuple))